<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/field_sim_2d_phi4_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# field_sim_2d_phi4.py

import os
import json
import math
import time
import argparse
from dataclasses import dataclass, asdict
from typing import Literal, Tuple

import numpy as np
import matplotlib.pyplot as plt


# ----------------------------
# Configuration dataclass
# ----------------------------
@dataclass
class Config:
    # Grid / time
    nx: int = 256
    ny: int = 256
    dx: float = 1.0
    dt: float = 0.1
    steps: int = 5000

    # Physics
    c: float = 1.0          # wave speed
    lam: float = 1.0        # lambda
    v: float = 1.0          # vacuum expectation value
    eta: float = 0.0        # uniform damping
    absorb_width: int = 0   # extra damping ramp near edges (0 disables)
    absorb_eta: float = 0.1 # max extra damping at boundary (blends to 0 inside)

    # Initialization
    init: Literal["random", "blob", "bias"] = "random"
    random_amp: float = 0.01
    blob_sigma: float = 20.0     # controls blob width (in grid units)
    bias_value: float = 0.9

    # Output / logging
    out_dir: str = "out_phi4"
    save_every: int = 50
    cmap: str = "seismic"
    seed: int = 42


# ----------------------------
# Utilities
# ----------------------------
def ensure_dir(p: str) -> None:
    os.makedirs(p, exist_ok=True)

def save_json(path: str, obj) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)


# ----------------------------
# Physics helpers
# ----------------------------
def five_point_laplacian(phi: np.ndarray, dx: float) -> np.ndarray:
    """Periodic 5-point Laplacian."""
    return (
        np.roll(phi, +1, axis=0) +
        np.roll(phi, -1, axis=0) +
        np.roll(phi, +1, axis=1) +
        np.roll(phi, -1, axis=1) -
        4.0 * phi
    ) / (dx * dx)

def central_gradients(phi: np.ndarray, dx: float) -> Tuple[np.ndarray, np.ndarray]:
    """Central-difference gradients (periodic)."""
    gx = (np.roll(phi, -1, axis=0) - np.roll(phi, +1, axis=0)) / (2.0 * dx)
    gy = (np.roll(phi, -1, axis=1) - np.roll(phi, +1, axis=1)) / (2.0 * dx)
    return gx, gy

def potential(phi: np.ndarray, lam: float, v: float) -> np.ndarray:
    return 0.25 * lam * (phi * phi - v * v) ** 2

def dV_dphi(phi: np.ndarray, lam: float, v: float) -> np.ndarray:
    return lam * phi * (phi * phi - v * v)

def build_absorb_mask(nx: int, ny: int, width: int, max_eta: float) -> np.ndarray:
    """Create a smooth 2D damping ramp at the edges. 0 in interior, up to max_eta at boundary."""
    if width <= 0 or max_eta <= 0:
        return np.zeros((nx, ny), dtype=np.float32)
    x = np.arange(nx)
    y = np.arange(ny)
    xx, yy = np.meshgrid(x, y, indexing="ij")
    # Distance to nearest boundary (in grid cells)
    dist = np.minimum.reduce([xx, yy, nx - 1 - xx, ny - 1 - yy])
    # Ramp: 0 in interior; rises to 1 at the boundary across 'width' cells
    ramp = np.clip(1.0 - dist / width, 0.0, 1.0)
    # Smooth (cosine) ramp to avoid reflections
    smooth = 0.5 * (1.0 - np.cos(np.pi * np.clip(ramp, 0, 1)))
    return (max_eta * smooth).astype(np.float32)


# ----------------------------
# Simulator
# ----------------------------
class FieldSimulator:
    def __init__(self, cfg: Config):
        self.cfg = cfg
        self.nx, self.ny = cfg.nx, cfg.ny
        self.dx, self.dt = cfg.dx, cfg.dt
        self.c, self.lam, self.v = cfg.c, cfg.lam, cfg.v
        self.eta = cfg.eta
        self.absorb = build_absorb_mask(cfg.nx, cfg.ny, cfg.absorb_width, cfg.absorb_eta)

        # Fields
        self.phi = np.zeros((self.nx, self.ny), dtype=np.float32)
        self.phi_prev = np.zeros_like(self.phi)
        self._initialize()

        # CFL check for 2D 5-point stencil: c*dt/dx <= 1/sqrt(2) recommended
        self._check_cfl()

    def _check_cfl(self):
        s = self.c * self.dt / self.dx
        s_max = 1.0 / math.sqrt(2.0)
        if s > s_max:
            print(f"[WARN] CFL condition likely violated: c*dt/dx = {s:.3f} > {s_max:.3f} (2D). "
                  f"Expect instabilities. Consider reducing dt or c, or increasing dx.")
        else:
            print(f"[INFO] CFL OK: c*dt/dx = {s:.3f} <= {s_max:.3f}")

    def _initialize(self):
        cfg = self.cfg
        rng = np.random.default_rng(cfg.seed)
        if cfg.init == "random":
            self.phi[:] = cfg.random_amp * (rng.random((self.nx, self.ny)) - 0.5)
        elif cfg.init == "blob":
            # Proper ij indexing so shapes match (nx, ny)
            i = np.arange(self.nx)
            j = np.arange(self.ny)
            ii, jj = np.meshgrid(i, j, indexing="ij")
            r2 = (ii - (self.nx - 1) / 2.0) ** 2 + (jj - (self.ny - 1) / 2.0) ** 2
            self.phi[:] = np.exp(-r2 / (2.0 * cfg.blob_sigma * cfg.blob_sigma))
        elif cfg.init == "bias":
            self.phi[:] = cfg.bias_value
        else:
            raise ValueError(f"Unknown init: {cfg.init}")

        # Start at rest (phi_t ~ 0): set previous via backward Euler step
        self.phi_prev[:] = self.phi.copy()

    def step(self):
        # Discrete leapfrog with damping. Approx phi_t ≈ (phi - phi_prev)/dt
        lap = five_point_laplacian(self.phi, self.dx)
        force = (self.c * self.c) * lap - dV_dphi(self.phi, self.lam, self.v)

        # Effective damping field (global eta + edge absorption)
        eta_eff = self.eta + self.absorb
        # Damping term ~ -eta * phi_t. In leapfrog, include via:
        # phi_new = (2 - eta*dt) * phi - (1 - eta*dt) * phi_prev + dt^2 * force
        # This is a simple, stable discretization that damps velocity.
        damp = eta_eff * self.dt
        phi_new = ((2.0 - damp) * self.phi) - ((1.0 - damp) * self.phi_prev) + (self.dt * self.dt) * force

        self.phi_prev, self.phi = self.phi, phi_new

    def energy(self) -> float:
        # Kinetic: 0.5 * (phi_t)^2, with phi_t ≈ (phi - phi_prev)/dt
        phi_t = (self.phi - self.phi_prev) / self.dt
        kin = 0.5 * (phi_t * phi_t)

        # Gradient: 0.5 * c^2 * |grad phi|^2 (central)
        gx, gy = central_gradients(self.phi, self.dx)
        grad = 0.5 * (self.c * self.c) * (gx * gx + gy * gy)

        # Potential
        pot = potential(self.phi, self.lam, self.v)

        e_density = kin + grad + pot
        return float(e_density.sum() * self.cfg.dx * self.cfg.dx)


# ----------------------------
# Visualization
# ----------------------------
def plot_field(phi: np.ndarray, step: int, out_dir: str, cmap: str):
    ensure_dir(out_dir)
    plt.figure(figsize=(6, 6))
    # Transpose only for conventional x-right, y-up display
    im = plt.imshow(phi.T, cmap=cmap, origin="lower")
    plt.colorbar(im, fraction=0.046, pad=0.04, label="φ")
    plt.title(f"Field φ at step {step}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.tight_layout()
    path = os.path.join(out_dir, f"field_{step:06d}.png")
    plt.savefig(path, dpi=120)
    plt.close()


# ----------------------------
# Runner
# ----------------------------
def run(cfg: Config):
    t0 = time.time()
    out = cfg.out_dir
    frames_dir = os.path.join(out, "frames")
    ensure_dir(out)
    ensure_dir(frames_dir)

    # Save config
    save_json(os.path.join(out, "config.json"), asdict(cfg))

    sim = FieldSimulator(cfg)

    # Energy log
    energy_path = os.path.join(out, "energy.csv")
    with open(energy_path, "w", encoding="utf-8") as f:
        f.write("step,energy\n")

    print("[INFO] Starting simulation...")
    for step in range(cfg.steps + 1):
        if step % cfg.save_every == 0:
            E = sim.energy()
            plot_field(sim.phi, step, frames_dir, cfg.cmap)
            with open(energy_path, "a", encoding="utf-8") as f:
                f.write(f"{step},{E:.6e}\n")
            print(f"Step {step:6d} | Energy = {E:.6e}")

        if step < cfg.steps:
            sim.step()

    dt = time.time() - t0
    print(f"[DONE] Completed {cfg.steps} steps in {dt:.2f}s. Outputs in: {out}")


# ----------------------------
# CLI (notebook-safe)
# ----------------------------
def build_parser():
    p = argparse.ArgumentParser(
        description="2D φ⁴ field simulator with leapfrog time integration, energy tracking, and damping."
    )
    # Grid/time
    p.add_argument("--nx", type=int, default=256)
    p.add_argument("--ny", type=int, default=256)
    p.add_argument("--dx", type=float, default=1.0)
    p.add_argument("--dt", type=float, default=0.1)
    p.add_argument("--steps", type=int, default=5000)

    # Physics
    p.add_argument("--c", type=float, default=1.0)
    p.add_argument("--lam", type=float, default=1.0)
    p.add_argument("--v", type=float, default=1.0)
    p.add_argument("--eta", type=float, default=0.0)
    p.add_argument("--absorb_width", type=int, default=0)
    p.add_argument("--absorb_eta", type=float, default=0.1)

    # Init
    p.add_argument("--init", choices=["random", "blob", "bias"], default="random")
    p.add_argument("--random_amp", type=float, default=0.01)
    p.add_argument("--blob_sigma", type=float, default=20.0)
    p.add_argument("--bias_value", type=float, default=0.9)

    # Output
    p.add_argument("--out_dir", type=str, default="out_phi4")
    p.add_argument("--save_every", type=int, default=50)
    p.add_argument("--cmap", type=str, default="seismic")
    p.add_argument("--seed", type=int, default=42)
    return p

def main(argv=None):
    parser = build_parser()
    # Notebook-safe: ignore stray -f kernel.json
    args, _ = parser.parse_known_args(argv)
    cfg = Config(**vars(args))
    run(cfg)

if __name__ == "__main__":
    main()